In [7]:
import numpy as np

In [8]:
class Storage():
    def __init__(self):
        self.storage = dict()

    def append_val(self, key, val):
        try:
            self.storage[key].append(val)
        except:
            self.storage[key] = [val]

    def get_values(self, key):
        try:
            return self.storage[key]
        except:
            return []
        

class LSH():
    def __init__(self, bits_number, embedding_dimention, hashtables_number=1):
        self.bits_number = bits_number
        self.embedding_dimention = embedding_dimention
        self.hashtables_number = hashtables_number
        self.hash_tables = [Storage() for i in range(self.hashtables_number)]
        self.planes = [np.random.randn(self.bits_number, self.embedding_dimention)
                       for _ in range(self.hashtables_number)]
    
    def _hash(self, planes, point):
        result = []
        for val in np.dot(planes, np.array(point)):
            if val > 0:
                result.append('1')
            else:
                result.append('0')
        return "".join(result)

    def add_to_hash_table(self, point):
        for i in range(len(self.hash_tables)):
            self.hash_tables[i].append_val(self._hash(self.planes[i], point), tuple(point))

    def find_n_similar(self, point, dist_func, n=1):
        candidates = []
        for i in range(len(self.hash_tables)):
            point_hash = self._hash(self.planes[i], point)
            candidates += self.hash_tables[i].get_values(point_hash)
        candidates = [(candidate, dist_func(point, np.array(list(candidate)).astype(int))) for candidate in candidates]
        candidates = sorted(candidates, key=lambda x: x[1])
        return [x[0] for x in candidates][:n]

def l2(x, y):
    return np.sum((x - y) ** 2)

In [9]:
a = LSH(bits_number=20, embedding_dimention=30, hashtables_number=2)

In [10]:
for _ in range(50000):
    a.add_to_hash_table([np.random.random() for _ in range(30)])

In [11]:
b = a.find_n_similar([np.random.random() for _ in range(30)], dist_func=l2, n=3)